In [1]:
import numpy as np
import pandas as pd 
import jieba 
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from collections import Counter
from math import isnan
from sklearn.feature_extraction.text import TfidfTransformer
#txt = pd.read_excel('数据比较好的.xlsx')
txt1 = pd.read_excel('../guo(new_3w).xlsx').drop('空白字段',axis = 1)
txt2 = pd.read_excel('../数据比较好的.xlsx')
txt = pd.concat([txt1,txt2],axis = 0)
txt = txt.drop_duplicates(subset=['商品描述'], keep='first')
txt = txt.reset_index(drop = True)
Y = txt['标签']

c:\python3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
for i in range(len(txt)):
    length = len(txt.loc[i]['商品描述'].split())
    if length >15:
        txt.drop(index = i,inplace=True)

In [3]:
frame = pd.DataFrame(columns=['商品描述','标签'])
dic = Counter(Y)
for i in dic.keys():
    data = txt.loc[txt['标签'] == i]
    """if dic[i] > 300:
        frame = pd.concat([frame,data[:300]],axis = 0)
        continue
    elif dic[i]>=30:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)"""
    if dic[i]>= 20:
        frame = pd.concat([frame,txt.loc[txt['标签'] == i]],axis = 0)
frame = frame.reset_index(drop=True)
c_num = len(Counter(frame['标签']).keys())
print('数据量：%d'%frame.shape[0],'分类数目：%d'%len(Counter(frame['标签']).keys()))

数据量：89778 分类数目：954


In [4]:
word_txt = []
for label in Counter(frame['标签']).keys():
    same = frame.loc[frame['标签'] == label]
    word = []
    for i in same['商品描述']:
        word += i.split()
    word_txt.append(word)
from gensim.models import word2vec
#sentences = word2vec.Text8Corpus('分词拿去做word2vect.txt')
model = word2vec.Word2Vec(word_txt,size = 200,window = 2000,min_count = 10)

c:\python3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
token_dictionary = {}
token_index = 0
word2vec_matrix = []
for line in range(len(frame['商品描述'])):
    tokens = frame['商品描述'][line].split()
    del_list = []
    for token in tokens:
        try:
            temp = model[token]
        except:
            del_list.append(token)
            continue
        else:
            if token not in token_dictionary:
                word2vec_matrix.append(list(temp))
                token_dictionary[token] = token_index
                token_index += 1
    for token in del_list:
        tokens.remove(token)
    frame['商品描述'][line] = ' '.join(tokens)
word2vec_matrix.append([0]*200)
token_dictionary['</s>'] = token_index

c:\python3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [6]:
frame_index = pd.DataFrame(columns=['商品描述','标签'])
padding_index = len(token_dictionary) - 1
index = []
for line in frame['商品描述']:
    tokens = line.split()
    line_index = []
    for token in tokens:
        line_index += [token_dictionary[token]]
    for _ in range(len(line_index),15):
        line_index.append(padding_index)
    index.append(line_index)
frame_index['商品描述'] = pd.Series(index)
frame_index['标签'] = frame['标签']

In [7]:
from sklearn import preprocessing 
le = preprocessing.LabelEncoder()
y_label = le.fit_transform(frame_index['标签'])

In [8]:
import tensorflow as tf
from tensorflow.contrib import rnn
import random
input_size = 200
timestep_size = 15
hidden_size = 256
layer_num = 2
class_num = c_num
learning_rate = 0.01
embed_dim = 200
sequence_length = 15
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    batch_size = tf.placeholder(tf.int32, [], name='batch_size_input')
    #_X = tf.placeholder(tf.float32,[None,1500])
    y = tf.placeholder(tf.int64,[None])
    #x = tf.reshape(_X,[-1,timestep_size,input_size])
    input_data = tf.placeholder(tf.int32, shape=[None, sequence_length], name='input_data') 
with tf.name_scope("embedding_layer"):
    #embedding = tf.get_variable("embedding",[vocabulary_size,embed_dim],dtype=tf.float32)
    embedding = tf.Variable(word2vec_matrix,dtype = tf.float32)
    inputs=tf.nn.embedding_lookup(embedding,input_data)    
    
#inputs = tf.nn.dropout(inputs, keep_prob)
#input_data (batch_size , sequence_length)
#inputs shape : (batch_size , sequence_length , embed_dim)

inputs = tf.transpose(inputs, [1,0,2])
#inputs shape : (sequence_length, batch_size, embed_dim)

inputs = tf.reshape(inputs, [-1, embed_dim])

# 转换成list,里面的每个元素是(batch_size, embed_dim)
inputs = tf.split(inputs, sequence_length, 0)

with tf.name_scope("fw"):
    stacked_rnn_fw = []
    for _ in range(layer_num):
        #fw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        fw_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
        stacked_rnn_fw.append(fw_cell)
    lstm_fw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_fw), output_keep_prob= keep_prob)
    #lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)
with tf.name_scope("bw"):
    stacked_rnn_bw = []
    for _ in range(layer_num):
        #bw_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size, forget_bias=1.0, state_is_tuple=True)
        bw_cell = tf.nn.rnn_cell.GRUCell(hidden_size)
        stacked_rnn_bw.append(bw_cell)
    lstm_bw_cell_m = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(stacked_rnn_bw), output_keep_prob= keep_prob)
    #lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
with tf.name_scope("output"):
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, inputs, dtype=tf.float32)

In [9]:
with tf.variable_scope("result",reuse = tf.AUTO_REUSE):
    fc_w = tf.Variable(tf.truncated_normal([2*hidden_size , class_num], stddev=0.1), name='fc_w')
    fc_b = tf.Variable(tf.zeros([class_num]), name='fc_b')
    #self.final_output = outputs[-1]
    # 用于分类任务, outputs取最终一个时刻的输出
    #logits = tf.matmul(final_output, fc_w) + fc_b
    logits = tf.matmul(outputs[-1], fc_w) + fc_b
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits+1e-10,labels = y)
    original_cost_function = tf.reduce_mean(loss)
    
    tv = tf.trainable_variables()
    regularization_cost = 0.001* tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv ])
    cost = original_cost_function + regularization_cost

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(original_cost_function)
with tf.name_scope("accuracy"):
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction,y)
    correct_num=tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")

In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(frame_index['商品描述'].tolist(),y_label,test_size = 0.2)

In [11]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    listnum = []
    Iter = 0
    for i in range(600):
        _batch_size = 500
        if len(listnum)<_batch_size:
            listnum = list(range(len(X_train)))
            Iter += 1
        temp_x = []
        temp_y = []
        for k in range(_batch_size):
            j = random.randint(0,len(listnum)-1) #生成一个包括0，len(listnum)-1之间的随机数
            temp_x.append(X_train[listnum[j]])
            temp_y.append(Y_train[listnum[j]])
            del listnum[j]
        # temp_x 是一个1000*1200维度的向量
        if (i)%20 == 0:
            train_accuracy_test = sess.run(accuracy, feed_dict={input_data:X_test,y: Y_test, keep_prob: 1.0, batch_size:len(X_test)})
            train_accuracy_train = sess.run(accuracy, feed_dict={input_data:temp_x,y: temp_y, keep_prob: 1.0, batch_size:_batch_size})
            print("Iter %d: step %d, training accuracy %g" % ( Iter,(i+1), train_accuracy_test))
            print('训练集表现：',train_accuracy_train)
        sess.run(train_op,feed_dict={input_data:temp_x,y:temp_y,keep_prob: 0.3, batch_size: _batch_size})

Iter 1: step 1, training accuracy 0.00445534
训练集表现： 0.002
Iter 1: step 21, training accuracy 0.540488
训练集表现： 0.5
Iter 1: step 41, training accuracy 0.646915
训练集表现： 0.66
Iter 1: step 61, training accuracy 0.705057
训练集表现： 0.682
Iter 1: step 81, training accuracy 0.738862
训练集表现： 0.718
Iter 1: step 101, training accuracy 0.765984
训练集表现： 0.78
Iter 1: step 121, training accuracy 0.785476
训练集表现： 0.782
Iter 1: step 141, training accuracy 0.797394
训练集表现： 0.816
Iter 2: step 161, training accuracy 0.803297
训练集表现： 0.836
Iter 2: step 181, training accuracy 0.820172
训练集表现： 0.834
Iter 2: step 201, training accuracy 0.827467
训练集表现： 0.808
Iter 2: step 221, training accuracy 0.83532
训练集表现： 0.842
Iter 2: step 241, training accuracy 0.840666
训练集表现： 0.818
Iter 2: step 261, training accuracy 0.840555
训练集表现： 0.84
Iter 2: step 281, training accuracy 0.847071
训练集表现： 0.862
Iter 3: step 301, training accuracy 0.85264
训练集表现： 0.876
Iter 3: step 321, training accuracy 0.854143
训练集表现： 0.89
Iter 3: step 341, training

In [12]:
frematrix = CountVectorizer().fit_transform(frame['商品描述'])
tfidf = TfidfVectorizer().fit_transform(frame['商品描述'])
lr = LogisticRegression()
X_train,X_test,Y_train,Y_test = train_test_split(tfidf,frame['标签'],test_size = 0.2)
lr.fit(X_train,Y_train)
pre = lr.predict(X_test)
np.mean(Y_test == pre)

0.8542548451770996

In [13]:
a = [int(i) for i in input().split()]
print(a[0] + a[1])

1


KeyboardInterrupt: 

In [ ]:
while True:
    try:
        a,b = map(int,input().split())
        print(a+b)
    except:
        break;